[![colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/davemlz/eemont/blob/master/docs/tutorials/023-Creating-Geometries-From-Plus-Codes.ipynb)
[![Open in SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/davemlz/eemont/blob/master/docs/tutorials/023-Creating-Geometries-From-Plus-Codes.ipynb)
[![Open in Planetary Computer](https://img.shields.io/badge/Open-Planetary%20Computer-black?style=flat&logo=microsoft)](https://pccompute.westeurope.cloudapp.azure.com/compute/hub/user-redirect/git-pull?repo=https://github.com/davemlz/eemont&urlpath=lab/tree/eemont/docs/tutorials/023-Creating-Geometries-From-Plus-Codes.ipynb&branch=master)

# Creating Geometries From Plus Codes (and Vice Versa)

_Tutorial created by **Aaron Zuspan**_: [GitHub](https://github.com/aazuspan) | [Twitter](https://twitter.com/aazuspan)

- GitHub Repo: [https://github.com/davemlz/eemont](https://github.com/davemlz/eemont)
- PyPI link: [https://pypi.org/project/eemont/](https://pypi.org/project/eemont/)
- Conda-forge: [https://anaconda.org/conda-forge/eemont](https://anaconda.org/conda-forge/eemont)
- Documentation: [https://eemont.readthedocs.io/](https://eemont.readthedocs.io/)
- More tutorials: [https://github.com/davemlz/eemont/tree/master/docs/tutorials](https://github.com/davemlz/eemont/tree/master/docs/tutorials)

## Let's start!

If required, please uncomment:

In [1]:
#!pip install eemont
#!pip install geemap
#!pip install openlocationcode

Import the required packages.

In [2]:
import ee, eemont, geemap

Authenticate and Initialize Earth Engine and geemap.

In [3]:
Map = geemap.Map()

## Creating a Single Point  (ee.Geometry)

A [Plus Code](https://maps.google.com/pluscodes/) represents the coordinates of a small area. Creating a point from a Plus Code will return the centroid geometry of that area. 

### From a Full Plus Code

Plus Codes can be full or short. Full Plus Codes contain all of the information needed to retrieve a location:

In [4]:
fullCode = '85FPQXGV+XH'

An ee.Geometry.Point can be constructed from a full Plus Code using the ee.Geometry.PointFromPlusCode constructor (extended through eemont):

In [5]:
pointFromFullPlusCode = ee.Geometry.PointFromPlusCode(fullCode)

Let's visualize out point (color blue):

In [6]:
Map.addLayer(pointFromFullPlusCode,{'color':'blue'},'Full')
Map.centerObject(pointFromFullPlusCode,10)
Map

Map(center=[39.77743750000001, -105.00606250000001], controls=(WidgetControl(options=['position', 'transparent…

### From a Short Plus Code

Full Plus Codes can be turned into short Codes by removing characters from the beginning of the Code and adding a nearby reference location to the end of the Code.

In [7]:
shortCode = 'QXGV+XH Denver, CO, USA'

Short Plus Codes can be more readable than full Codes, but they must be geocoded internally before use. In [eemont](https://github.com/davemlz/eemont), reference locations are geocoded using the [geopy](https://geopy.readthedocs.io/en/latest/#) package.

> NOTE: Neither eemont nor geopy are geocoding services.

An ee.Geometry.Point can also be constructed from a short Plus Code using the ee.Geometry.PointFromPlusCode constructor (extended through eemont):

In [8]:
pointFromShortPlusCode = ee.Geometry.PointFromPlusCode(shortCode, user_agent = 'eemont-tutorial-023')

Unlike with a full Plus Code, the `user_agent` argument must be specified: This is a string describing the name of the app that is using a geocoding service (you can use here your GEE username).

Let's visualize our point (color yellow). Notice that it represents the same location as the full Plus Code above.

In [9]:
Map.addLayer(pointFromShortPlusCode,{'color':'yellow'},'Short')
Map.centerObject(pointFromShortPlusCode,10)
Map

Map(center=[39.77743750000001, -105.00606250000001], controls=(WidgetControl(options=['position', 'transparent…

By default, the geocoding service used is `nominatim` ([Open Street Maps](https://nominatim.openstreetmap.org/ui/search.html)). But it can be modified using the `geocoder` parameter (let's use the arcgis geocoding service):

In [10]:
pointFromShortPlusCode = ee.Geometry.PointFromPlusCode(shortCode, geocoder = 'arcgis', user_agent = 'eemont-tutorial-023')

## Creating Other Geometries (ee.Geometry)

More complex geometries can be created from lists of full or short Plus Codes, just as they can be created from lists of latitude and longitude coordinates. The following Plus Code geometry constructors are implemented by eemont:
- ee.Geometry.PointFromPlusCode
- ee.Geometry.MultiPointFromPlusCodes
- ee.Geometry.PolygonFromPlusCodes
- ee.Geometry.MultiPolygonFromPlusCodes
- ee.Geometry.LineStringFromPlusCodes
- ee.Geometry.MultiLineStringFromPlusCodes
- ee.Geometry.LinearRingFromPlusCodes
- ee.Geometry.RectangleFromPlusCodes

Let's start with a list of Plus Codes and see how we can use them to create different geometries:

In [11]:
codes = ['85FQ2222+22', '85FR2222+22', '85GR2222+22']

The Plus Codes above represent 3 points. We can turn those 3 points into a MultiPoint:

In [12]:
multiPointFromPlusCodes = ee.Geometry.MultiPointFromPlusCodes(codes)
Map.addLayer(multiPointFromPlusCodes, {"color": "red"}, "MultiPoint")
Map.centerObject(multiPointFromPlusCodes, 8)
Map

Map(center=[39.334345067804215, -104.33486191769688], controls=(WidgetControl(options=['position', 'transparen…

Or a LineString:

In [13]:
lineStringFromPlusCodes = ee.Geometry.LineStringFromPlusCodes(codes)
Map.addLayer(lineStringFromPlusCodes, {"color": "red"}, "LineString")
Map.centerObject(lineStringFromPlusCodes, 8)
Map

Map(center=[39.28214272083444, -104.21946298377237], controls=(WidgetControl(options=['position', 'transparent…

Or a Polygon:

In [14]:
polygonFromPlusCodes = ee.Geometry.PolygonFromPlusCodes(codes)
Map.addLayer(polygonFromPlusCodes, {"color": "red"}, "Polygon")
Map.centerObject(polygonFromPlusCodes, 8)
Map

Map(center=[39.33434976499757, -104.33486261811156], controls=(WidgetControl(options=['position', 'transparent…

A nested list of lists of Plus Codes can be used to create more complex geometries.

In [15]:
nestedCodes = [
    ['85FQ2222+22', '85FR2222+22', '85GR2222+22'], 
    ['85FP8PC2+G2', '85FPJF23+G4', '85FPMW2R+RP'],
    ['85FPRJ5W+82', '85GP3M67+CP', '85GQ2R7C+38'],
]

For example, we can use nested Plus Codes to create a MultiLineString:

In [16]:
multiLineStringFromPlusCodes = ee.Geometry.MultiLineStringFromPlusCodes(nestedCodes)
Map.addLayer(multiLineStringFromPlusCodes, {"color": "red"}, "MultiLineString")
Map.centerObject(multiLineStringFromPlusCodes, 8)
Map

Map(center=[39.56571884825117, -104.65319603966626], controls=(WidgetControl(options=['position', 'transparent…

Or a MultiPolygon:

In [17]:
multiPolygonFromPlusCodes = ee.Geometry.MultiPolygonFromPlusCodes(nestedCodes)
Map.addLayer(multiPolygonFromPlusCodes, {"color": "red"}, "MultiPolygon")
Map.centerObject(multiPolygonFromPlusCodes, 8)
Map

Map(center=[39.33434976499757, -104.33486261811156], controls=(WidgetControl(options=['position', 'transparent…

## Retrieving Plus Codes From Geometries or Features

The coordinates of any ee.Geometry or ee.Feature can be retrieved as Plus Codes, regardless of whether it was created with Plus Codes or not. For example, let's [create a point from a query](https://colab.research.google.com/github/davemlz/eemont/blob/master/tutorials/010-Creating-Points-From-Queries.ipynb) and extract it's Plus Code equivalent:

In [18]:
queryPoint = ee.Geometry.PointFromQuery('London, England', user_agent='eemont-tutorial-023')
queryPlusCode = queryPoint.plusCodes()

queryPlusCode

'9C3XGV4C+WW'

Let's create a Point from that Plus Code to ensure that it worked correctly:

In [19]:
pointFromQueryPlusCode = ee.Geometry.PointFromPlusCode(queryPlusCode)
Map.addLayer(pointFromQueryPlusCode, {"color": "green"}, "Query Point")
Map.centerObject(pointFromQueryPlusCode, 8)
Map

Map(center=[51.5073125, -0.1276875], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HB…